In [1]:
import os 
import time
import requests  
import json  
import openai
import numpy as np
from openai import OpenAI
#from prompt_construction import read_prompt_list ,  prompt_construction module is not available here

api_key = os.environ["OPENAI_API_KEY"]  

%reload_ext autoreload
%autoreload 2

In [2]:
# Define the API endpoint URL  
def get_response_siemens_api(url, model, max_token, user_content, system_content):  
    # Define the request headers  
    headers = {  
        'accept': 'application/json',  
        'Authorization': 'Bearer {}'.format(api_key),  
        'Content-Type': 'application/json'  
    }  

    # Define the request payload  
    payload = {  
        'model': model,  
        'messages': [  
            {'role': 'user', 'content': user_content },
            {'role': 'system', 'content': system_content}           
        ],  
        'temperature': 0,  
        'stream': False,  
        'max_tokens': max_token          
    }  
      
    # Make the POST request  
    response = requests.post(url, headers=headers, data=json.dumps(payload))  
      
    # Print the response  
    return print(response.json()["choices"][0]["message"]["content"])


In [3]:
url = 'https://api.siemens.com/llm/chat/completions'  
model = 'mistral-7b-instruct'
max_token = 100
user_content = ("Please classify the given requirement into 'functional requirement' or 'non-functional requirement'. "
                "The answer should be in format {the given requirement: functional requirement or non-functional requirement}."
                "The given requirement: The system shall refresh the display every 60 seconds." )
system_content = ("You are a senior software engineer who is experienced in software requirement classification! " )

get_response_siemens_api(url, model, max_token, user_content, system_content)

 {The system shall refresh the display every 60 seconds: functional requirement}


# define useful functions

In [33]:
# Define the API endpoint URL  
def get_response_siemens_api(prompt):  
    # Define the request headers  
    headers = {  
        'accept': 'application/json',  
        'Authorization': 'Bearer {}'.format(api_key),  
        'Content-Type': 'application/json'  
    }  

    # Define the request payload  
    payload = {  
        'model': 'mistral-7b-instruct',  
        'messages': [  
            {'role': 'user', 'content': prompt },
            {'role': 'system', 'content': "You are a senior software engineer who is experienced in software requirement classification! "}           
        ],  
        'temperature': 0,  
        'stream': False,  
        'max_tokens': 6666          
    }  
      
    # Make the POST request  
    url = 'https://api.siemens.com/llm/chat/completions'  
    response = requests.post(url, headers=headers, data=json.dumps(payload))  
      
    # Print the response  
    return response.json()["choices"][0]["message"]["content"]


In [49]:
def get_completion(prompt_list, save_path):
    """load the prompt_list, get the completion from llm, and save the results."""

    start_time = time.time()  
    
    completion_list = []
    for i, prompt in enumerate(prompt_list):
        #print("The prompt: " + prompt + "\n######")
     
        j = 0
        while j < 50:
            try:   
                completion = get_response_siemens_api(prompt)
                #print(completion)
                completion_list.append(completion)
                break
        
            except: #InvalidRequestError: in case that the prompt and output reach the token limitation
                #print("{}th prompt, Connection error: try again automatically".format(i) )
                time.sleep(5)  # stop 1 second to avoid error
            j = j+1   
            
        if j == 50:
            print("{}th prompt. At least one prompt did not get the result.".format(j))
            break

        # completion = get_response_siemens_api(prompt)
        # #print(completion)
        # completion_list.append(completion)
        # break
        


    save_completion_list(save_path, completion_list)

    # calculate the execution time
    end_time = time.time()  
    print("execution time: {}".format(end_time - start_time), "second")

In [50]:
def read_prompt_list(path):
    """read the saved list of prompts 
    input
        path, string path to save the list
    """
    with open(path, 'r') as file:  
        content = file.read()  
    prompt_list_read = content.split('\n\n\n')   

    return prompt_list_read


In [51]:
def save_completion_list(path, completion_list):
    """save the constructed prompts with few-shot examples in a list 
    input
        path, string path to save the list
        prompt_list, list of constructed prompts from the first requirement/sentence in test dataset to the last one
    """
    with open(path, 'w', newline='\n') as file:  
        for i, completion in enumerate(completion_list):
            if i+1 == len(completion_list):
                file.write(completion)
            else:     
                file.write(completion + "\n\n\n")

# get completion on promise dataset

## binary classification

In [52]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_0_bi.txt"
get_completion(prompt_list, save_path)

execution time: 100.27379727363586 second


In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_5_bi.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_10_bi.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_20_bi.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_40_bi.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_80_bi.txt"
get_completion(prompt_list, save_path)

#### binary embedding

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_0_bi.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_5_bi.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_10_bi.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_20_bi.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_40_bi.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_80_bi.txt"
get_completion(prompt_list, save_path)

#### binary tfidf

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_80.txt"
get_completion(prompt_list, save_path)

## multi-class classification

#### promise multi-class random

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_random_80.txt"
get_completion(prompt_list, save_path)

#### promise multiclass embedding

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_embedding_80.txt"
get_completion(prompt_list, save_path)

#### promise multiclass tf-idf

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_promise_tfidf_80.txt"
get_completion(prompt_list, save_path)

# get completion on nfr dataset

## multi-class classification

#### nfr multi_class random

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_random_80.txt"
get_completion(prompt_list, save_path)

#### nfr multi_class embedding

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_embedding_80.txt"
get_completion(prompt_list, save_path)

#### nfr multi_class tfidf

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/mistral_7b/completion_nfr_tfidf_80.txt"
get_completion(prompt_list, save_path)